<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/Coil3D_FrenetSerret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots
from functools import partial

In [17]:
theta=np.linspace(0,2*np.pi,36)
xy=np.array([np.cos(theta),np.sin(theta)])

In [18]:
xy_stretch=np.array([[2.,0.],[0.,1.]]) @ xy

rot_angle = np.pi/6
xy_stretch_rotate = np.array([[np.cos(rot_angle), -np.sin(rot_angle)],[np.sin(rot_angle), np.cos(rot_angle)]]) @ xy_stretch

In [19]:
fig=make_subplots()
fig.add_scatter(x=xy[0],y=xy[1],mode='markers')
fig.add_scatter(x=xy_stretch[0],y=xy_stretch[1],mode='markers')
fig.add_scatter(x=xy_stretch_rotate[0],y=xy_stretch_rotate[1],mode='markers')
fig.update_xaxes(range=[-3,3])
fig.update_yaxes(range=[-3,3])
fig.update_layout(width=600,height=600,template='plotly_dark')

In [110]:
t=jnp.linspace(0,4*np.pi,36)
theta = jnp.linspace(0,2*np.pi, 18)
R=3.
mz=0.4
r=0.5

In [97]:
def curve(t, R, mz):
    xyz = jnp.array([R*jnp.cos(t), R*jnp.sin(t), mz*t])
    return xyz

In [98]:
def frenet(f):

    dfdt = jax.jacobian(f)

    def dsdt(t):
        return jnp.linalg.norm(dfdt(t))

    def tangent(t):   #df_ds = dfdt / dsdt
        return dfdt(t)/dsdt(t)

    dtangent_dt = jax.jacobian(tangent)

    def normal(t):
        v= dtangent_dt(t) / dsdt(t)
        return v/jnp.linalg.norm(v)


    return tangent, normal

In [99]:
xyz= curve(t,R,mz)

In [100]:
fig=make_subplots(rows=1,cols=1,specs=[[{'type':'surface'}]])
fig.add_scatter3d(x=xyz[0,:],y=xyz[1,:],z=xyz[2,:],mode='lines',row=1,col=1)
fig.update_layout(width=600,height=600,template='plotly_dark')

In [93]:
tangent, normal = frenet(lambda t: curve(t, R=R,mz=mz))

def surf(t,theta):
    T = tangent(t)
    N = normal(t)
    B = jnp.cross(T,N)
    return curve(t, R, mz) + jnp.stack([N, B], axis=1) @ jnp.array([r*jnp.cos(theta), r*jnp.sin(theta)])


In [106]:
surf_vec = jnp.vectorize(surf, signature='(),()->(3)')

In [116]:
s1=surf_vec(t[:,None], theta[None,:])

In [126]:
a = jnp.arange(5)
b= jnp.arange(10)
a[:,None]*b[None,:]

DeviceArray([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
             [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
             [ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18],
             [ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27],
             [ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36]], dtype=int64)

In [119]:
x,y,z = s1.T

In [120]:
x.shape

(18, 36)

In [127]:
fig=make_subplots(rows=1,cols=1,specs=[[{'type':'surface'}]])

for i in range(theta.size):
    fig.add_scatter3d(x=x[i,:],y=y[i,:],z=z[i,:],mode='lines', line_color='white',row=1,col=1)

for i in range(t.size):
    fig.add_scatter3d(x=x[:,i],y=y[:,i],z=z[:,i],mode='lines', line_color='white',row=1,col=1)

fig.update_layout(width=600,height=600,template='plotly_dark', showlegend=False)

In [9]:
def get_grad(f):

    def grad(x,h):
        res=[]
        xplush=np.copy(x)
        for i in range(x.size):
            xplush[i]=x[i]+h
            res.append((f(xplush) - f(x))/h)
        return np.array(res)
    return grad
        



In [10]:
def f(x):
    return np.sin(x[0])+ np.cos(2*x[1])

In [ ]:
grad=get_grad(f)

In [ ]:
grad(np.array([np.pi/6, np.pi/3]),h=1e-8)

array([ 0.8660254 , -0.86602539])